$\newcommand{\xv}{\mathbf{x}}
\newcommand{\Xv}{\mathbf{X}}
\newcommand{\yv}{\mathbf{y}}
\newcommand{\zv}{\mathbf{z}}
\newcommand{\av}{\mathbf{a}}
\newcommand{\Wv}{\mathbf{W}}
\newcommand{\wv}{\mathbf{w}}
\newcommand{\tv}{\mathbf{t}}
\newcommand{\Tv}{\mathbf{T}}
\newcommand{\muv}{\boldsymbol{\mu}}
\newcommand{\sigmav}{\boldsymbol{\sigma}}
\newcommand{\phiv}{\boldsymbol{\phi}}
\newcommand{\Phiv}{\boldsymbol{\Phi}}
\newcommand{\Sigmav}{\boldsymbol{\Sigma}}
\newcommand{\Lambdav}{\boldsymbol{\Lambda}}
\newcommand{\half}{\frac{1}{2}}
\newcommand{\argmax}[1]{\underset{#1}{\operatorname{argmax}}}
\newcommand{\argmin}[1]{\underset{#1}{\operatorname{argmin}}}$

# Assignment 6: Neural Networks

*Sean Russell*

## Overview

You will write and apply code that trains neural networks of various numbers of hidden layers and units in each hidden layer and returns results as specified below.  You will do this once for a regression problem and once for a classification problem. 

## Required Code

Download [nn2.tar](http://www.cs.colostate.edu/~anderson/cs440/notebooks/nn2.tar) that was used in lecture and extract its contents, which are

* `neuralnetworks.py`
* `scaledconjugategradient.py`
* `mlutils.py`

Write the following functions that train and evaluate neural network models.

* `results = trainNNs(X, T, trainFraction, hiddenLayerStructures, numberRepetitions, numberIterations, classify)`

The arguments to `trainNNs` are

* `X` is a matrix of input data of shape `nSamples x nFeatures`
* `T` is a matrix of target data of shape `nSamples x nOutputs`
* `trainFraction` is fraction of samples to use as training data. 1-`trainFraction` is number of samples for testing data
* `hiddenLayerStructures` is list of network architectures. For example, to test two networks, one with one hidden layer of 20 units, and one with 3 hidden layers with 5, 10, and 20 units in each layer, this argument would be `[[20], [5, 10, 20]]`.
* `numberRepetitions` is number of times to train a neural network.  Calculate training and testing average performance (two separate averages) of this many training runs.
* `numberIterations` is the number of iterations to run the scaled conjugate gradient algorithm when a neural network is trained.
* `classify` is set to `True` if you are doing a classification problem, in which case `T` must be a single column of target class integers.

This function returns `results` which is list with one element for each network structure tested.  Each element is a list containing 

* the hidden layer structure (as a list),
* a list of training data performance for each repetition, 
* a list of testing data performance for each repetition, and
* the number of seconds it took to run this many repetitions for this network structure.

This function should follow these steps:

  * For each network structure given in `hiddenLayerStructures`
    * For numberRepetitions
      * Use `ml.partition` to randomly partition X and T into training and testing sets.
      * Create a neural network of the given structure
      * Train it for numberIterations
      * Use the trained network to produce outputs for the training and for the testing sets
      * If classifying, calculate the fraction of samples incorrectly classified for training and testing sets.
       Otherwise, calculate the RMSE of training and testing sets.
      * Add the training and testing performance to a collection (such as a list) for this network structure
    * Add to a collection of all results the hidden layer structure, lists of training performance and testing performance, and seconds taken to do these repetitions.
  * return the collection of all results

Also write the following two functions. `summarize(results)` returns a list of lists like `results` but with the list of training performances replaced by their mean and the list of testing performances replaced by their mean.   
`bestNetwork(summary)` takes the output of `summarize(results)` and returns the best element of `results`, determined by the element that has the smallest test performance.

* `summary = summarize(results)` where `results` is returned by `trainNNs` and `summary` is like `results` with the training and testing performance lists replaced by their means
* `best = bestNetwork(summary)` where `summary` is returned by `summarize` and `best` is the best element of `summary`

Just a bunch of imports

In [4]:
import numpy as np
import mlutils as ml
import neuralnetworks as nn
import time
from sklearn.metrics import mean_squared_error
from math import sqrt
import copy
import pandas as pd

Heres the meat of the code. trainNNs trains and tests a bunch of networks all at once with different hidden layer structures and returns the performance of each different network structure. summarize takes these results and makes them a bit easier to read, and bestNetwork takes the summary and only returns the results for the best network structure.

In [5]:
def trainNNs(X, T, trainFraction, hiddenLayerStructures, numberRepetitions, numberIterations, classify=False):
    results = []
    for structure in hiddenLayerStructures:
        result = []
        trainingPerformance = []
        testingPerformance = []
        result.append(structure)
        startTime = time.time()
        for n in range(numberRepetitions):
            trainX, trainT, testX, testT = ml.partition(X,T,[trainFraction,1-trainFraction],classify)
            model = nn.NeuralNetworkClassifier(X.shape[1],structure,len(np.unique(T))) if classify else nn.NeuralNetwork(X.shape[1],structure,T.shape[1])
            model.train(trainX, trainT, numberIterations)
            predictedTrain = model.use(trainX)
            predictedTest = model.use(testX)
            RMSE = lambda actual, predicted: sqrt(mean_squared_error(actual, predicted))
            trainError = 1 - ml.percentCorrect(predictedTrain,trainT) / 100 if classify else RMSE(trainT, predictedTrain)
            testError = 1 - ml.percentCorrect(predictedTest,testT) / 100 if classify else RMSE(testT, predictedTest)
            trainingPerformance.append(trainError)
            testingPerformance.append(testError)
        endTime = time.time()
        result.append(trainingPerformance)
        result.append(testingPerformance)
        result.append(endTime - startTime)
        results.append(result)  
    return results

def summarize(results):
    results = copy.deepcopy(results)
    for result in results:
        result[1] = sum(result[1]) / len(result[1])
        result[2] = sum(result[2]) / len(result[2])
    return results

def bestNetwork(summary):
    bestResult = summary[0]
    for result in summary:
        if result[2] < bestResult[2]:
            bestResult = result
    return bestResult

## Data for Regression Experiment

From the UCI Machine Learning Repository, download the [Appliances energy prediction](http://archive.ics.uci.edu/ml/datasets/Appliances+energy+prediction) data.  You can do this by visiting the Data Folder for this data set, or just do this:

     !wget http://archive.ics.uci.edu/ml/machine-learning-databases/00374/energydata_complete.csv



Read this data into python.  One suggestion is to use the `loadtxt` function in the `numpy` package.  You may ignore the first column of each row which contains a data and time.  Also ignore the last two columns of random variables.  We will not use that in our modeling of this data.  You will also have to deal with the double quotes that surround every value in every field.  Read the first line of this file to get the names of the features.

Once you have read this in correctly, you should see values like this:

In [7]:
def readData(fileName, excludedColumns):
    df = pd.read_csv(fileName)
    for excluded in excludedColumns:
        df.drop(excluded, axis=1, inplace = True)
    return df

In [8]:
fileName = 'energydata_complete.csv'
excludedColumns = ['date', 'rv1', 'rv2']
df = readData(fileName, excludedColumns)
names = list(df.axes[1])
data = df.as_matrix()
Xenergy = data[:,2:]
Tenergy = data[:,:2]
Xnames = names[2:]
Tnames = names[:2]

Train several neural networks on all of this data for 100 iterations.  Plot the error trace (nnet.getErrorTrace()) to help you decide now many iterations might be needed.  100 may not be enough.  If for your larger networks the error is still decreasing after 100 iterations you should train all nets for more than 100 iterations.

Now use your `trainNNs`, `summarize`, and `bestNetwork` functions on this data to investigate various network sizes.

In [125]:
results = trainNNs(Xenergy, Tenergy, 0.8, [0, 5, [5, 5], [10, 10]], 10, 100)

In [126]:
summarize(results)

[[0, 67.14380511599147, 67.26430089723117, 2.2673590183258057],
 [5, 65.27477821593422, 65.585980240998, 23.882287979125977],
 [[5, 5], 64.87599148368307, 66.06982609562651, 35.216299057006836],
 [[10, 10], 62.90730436756324, 64.00417527576994, 58.27176809310913]]

In [127]:
bestNetwork(summarize(results))

[[10, 10], 62.90730436756324, 64.00417527576994, 58.27176809310913]

Test at least 10 different hidden layer structures.  Larger numbers of layers and units may do the best on training data, but not on testing data. Why?

Now train another network with your best hidden layer structure on 0.8 of the data and use the trained network on the testing data (the remaining 0.2 of the date).  As before use `ml.partition` to produce the training and testing sets.

For the testing data, plot the predicted and actual `Appliances` energy use, and the predicted and actual `lights` energy use, in two separate plots.  Discuss what you see.

## Data for Classification Experiment

From the UCI Machine Learning Repository, download the [Anuran Calls (MFCCs)](http://archive.ics.uci.edu/ml/datasets/Anuran+Calls+%28MFCCs%29) data.  You can do this by visiting the Data Folder for this data set, or just do this:

     !wget 'http://archive.ics.uci.edu/ml/machine-learning-databases/00406/Anuran Calls (MFCCs).zip'
     !unzip Anuran*zip
     
Read the data in the file `Frogs_MFCCs.csv` into python.  This will be a little tricky. Each line of the file is a sample of audio features plus three columns that label the sample by family, genus, and species. We will try to predict the species.  The tricky part is that the species is given as text.  We need to convert this to a target class, as an integer. The `numpy` function `unique` will come in handy here.

In [205]:
df = pd.read_csv('Frogs_MFCCs.csv')
exclude = ['Family', 'Genus', 'RecordID']
for excluded in exclude:
    df.drop(excluded, axis=1, inplace = True)
df

,MFCCs_ 1,MFCCs_ 2,MFCCs_ 3,MFCCs_ 4,MFCCs_ 5,MFCCs_ 6,MFCCs_ 7,MFCCs_ 8,MFCCs_ 9,MFCCs_10,...,MFCCs_14,MFCCs_15,MFCCs_16,MFCCs_17,MFCCs_18,MFCCs_19,MFCCs_20,MFCCs_21,MFCCs_22,Species
0,1.0,0.152936,-0.105586,0.200722,0.317201,0.260764,0.100945,-0.150063,-0.171128,0.124676,...,0.082245,0.135752,-0.024017,-0.108351,-0.077623,-0.009568,0.057684,0.118680,0.014038,AdenomeraAndre
1,1.0,0.171534,-0.098975,0.268425,0.338672,0.268353,0.060835,-0.222475,-0.207693,0.170883,...,0.022786,0.163320,0.012022,-0.090974,-0.056510,-0.035303,0.020140,0.082263,0.029056,AdenomeraAndre
2,1.0,0.152317,-0.082973,0.287128,0.276014,0.189867,0.008714,-0.242234,-0.219153,0.232538,...,0.050791,0.207338,0.083536,-0.050691,-0.023590,-0.066722,-0.025083,0.099108,0.077162,AdenomeraAndre
3,1.0,0.224392,0.118985,0.329432,0.372088,0.361005,0.015501,-0.194347,-0.098181,0.270375,...,-0.011567,0.100413,-0.050224,-0.136009,-0.177037,-0.130498,-0.054766,-0.018691,0.023954,AdenomeraAndre
4,1.0,0.087817,-0.068345,0.306967,0.330923,0.249144,0.006884,-0.265423,-0.172700,0.266434,...,0.037439,0.219153,0.062837,-0.048885,-0.053074,-0.088550,-0.031346,0.108610,0.079244,AdenomeraAndre
5,1.0,0.099704,-0.033408,0.349895,0.344535,0.247569,0.022407,-0.213767,-0.127916,0.277353,...,0.012486,0.180641,0.055242,-0.080487,-0.130089,-0.171478,-0.071569,0.077643,0.064903,AdenomeraAndre
6,1.0,0.021676,-0.062075,0.318229,0.380439,0.179043,-0.041667,-0.252300,-0.167117,0.220027,...,0.027070,0.216923,0.064853,-0.046620,-0.055146,-0.085972,-0.009127,0.065630,0.044040,AdenomeraAndre
7,1.0,0.145130,-0.033660,0.284166,0.279537,0.175211,0.005791,-0.183329,-0.158483,0.192567,...,-0.009015,0.184266,0.075654,-0.055978,-0.048219,-0.056637,-0.022419,0.070085,0.021419,AdenomeraAndre
8,1.0,0.271326,0.027777,0.375738,0.385432,0.272457,0.098192,-0.173730,-0.157857,0.207181,...,-0.044984,0.064425,-0.032167,-0.120723,-0.112607,-0.156933,-0.118527,-0.002471,0.002304,AdenomeraAndre
9,1.0,0.120565,-0.107235,0.316555,0.364437,0.307757,0.025992,-0.294179,-0.223236,0.268435,...,0.042678,0.236484,0.053436,-0.051073,-0.052568,-0.111338,-0.040014,0.090204,0.088025,AdenomeraAndre


In [169]:
np.reshape(Tanuran,(Tanuran.shape[0],1))

array([['AdenomeraAndre'],
       ['AdenomeraAndre'],
       ['AdenomeraAndre'],
       ..., 
       ['ScinaxRuber'],
       ['ScinaxRuber'],
       ['ScinaxRuber']], dtype=object)

In [187]:
len(np.unique(Tanuran))

10

In [200]:
names = list(df.axes[1])
data = df.as_matrix()
classes = np.unique(data[:,-1]).tolist()
classes.index("HylaMinuta")
Xanuran = data[:,:-1].astype(float)
Tanuran = data[:,-1].tolist()

for i, c in enumerate(Tanuran):
    Tanuran[i] = classes.index(c)
Tanuran = np.array(Tanuran)
Tanuran = np.reshape(Tanuran,(-1,1))
Xnames = names[:-1]
Tnames = names[-1:]

Xanuran

array([[  1.53e-01,  -1.06e-01,   2.01e-01, ...,   5.77e-02,   1.19e-01,
          1.40e-02],
       [  1.72e-01,  -9.90e-02,   2.68e-01, ...,   2.01e-02,   8.23e-02,
          2.91e-02],
       [  1.52e-01,  -8.30e-02,   2.87e-01, ...,  -2.51e-02,   9.91e-02,
          7.72e-02],
       ..., 
       [ -5.83e-01,  -3.43e-01,   2.95e-02, ...,   2.78e-02,  -5.31e-04,
         -8.04e-02],
       [ -5.19e-01,  -3.08e-01,  -4.92e-03, ...,   4.18e-02,  -2.79e-02,
         -9.69e-02],
       [ -5.09e-01,  -3.24e-01,   6.21e-02, ...,   3.16e-02,  -2.94e-02,
         -8.79e-02]])

In [194]:
Tanuran

array([[0],
       [0],
       [0],
       ..., 
       [9],
       [9],
       [9]])

In [177]:
Xanuran.shape, Tanuran.shape

((7195, 21), (7195, 1))

In [209]:
Xanuran[:2,:]

array([[ 0.15, -0.11,  0.2 ,  0.32,  0.26,  0.1 , -0.15, -0.17,  0.12,
         0.19, -0.08, -0.16,  0.08,  0.14, -0.02, -0.11, -0.08, -0.01,
         0.06,  0.12,  0.01],
       [ 0.17, -0.1 ,  0.27,  0.34,  0.27,  0.06, -0.22, -0.21,  0.17,
         0.27, -0.1 , -0.25,  0.02,  0.16,  0.01, -0.09, -0.06, -0.04,
         0.02,  0.08,  0.03]])

In [179]:
Tanuran[:2]

array([[0],
       [0]])

In [180]:
for i in range(10):
    print('{} samples in class {}'.format(np.sum(Tanuran==i), i))

672 samples in class 0
3478 samples in class 1
542 samples in class 2
310 samples in class 3
472 samples in class 4
1121 samples in class 5
270 samples in class 6
114 samples in class 7
68 samples in class 8
148 samples in class 9


In [214]:
results = trainNNs(Xanuran, Tanuran, 0.8, [0, 5, [5, 5]], 5, 100, classify=True)

In [215]:
summarize(results)

[[0, 0.030507296733842958, 0.035441278665740115, 3.209761142730713],
 [5, 0.035024322446143175, 0.044753300903405166, 7.202175140380859],
 [[5, 5], 0.043571924947880494, 0.054482279360667116, 8.477503061294556]]

In [216]:
bestNetwork(summarize(results))

[0, 0.030507296733842958, 0.035441278665740115, 3.209761142730713]

Now do an investigation like you did for the regression data. 

Test at least 10 different hidden layer structures. Then train another network with your best hidden layer structure on 0.8 of the data and use the trained network on the testing data (the remaining 0.2 of the date). 

Plot the predicted and actual `Species` for the testing data as an integer.  Discuss what you see.

## Grading

Download [A6grader.tar](http://www.cs.colostate.edu/~anderson/cs440/notebooks/A6grader.tar) and extract `A6grader.py` from it.

In [60]:
%run -i "A6grader.py"


Testing summarize([[[1,1], [1.2, 1.3, 1.4], [2.2, 2.3, 2.4], 0.5], [[2,2,2], [4.4, 4.3, 4.2], [6.5, 6.4, 6.3], 0.6]])

--- 10/10 points. Correctly returned [[[1, 1], 1.3, 2.3000000000000003, 0.5], [[2, 2, 2], 4.3, 6.3999999999999995, 0.6]]

Testing bestNetwork([[[1, 1], 1.3, 2.3, 0.5], [[2, 2, 2], 4.3, 1.3, 0.6]])

--- 10/10 points. Correctly returned [[2, 2, 2], 4.3, 1.3, 0.6]

X = np.random.uniform(-1, 1, (100, 3))
T = np.hstack(((X**2 - 0.2*X**3).sum(axis=1,keepdims=True),
               (np.sin(X)).sum(axis=1,keepdims=True)))
result = trainNNs(X, T, 0.7, [0, 5, 10, [20, 20]], 10, 100, False)

--- 20/20 points. Correct.

Testing bestNetwork(summarize(result))

--- 20/20 points. You correctly found that network [20, 20] is best.

A6 Execution Grade is 60/60

======================= The regression data set =======================

--- _/5 points. Read the data in energydata_complete.csv into variables Xenergy and Tenergy.

--- _/5 points. Train some networks by calling the NeuralNetw